In [43]:
%%file Q2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import dataset
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def mapper_get_words(self, _, desc):
        for word in WORD_RE.findall(desc):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        yield None, (sum(counts), word)

    def reducer_find_max_words(self, _, word_count_pairs):
        max_10 = sorted(word_count_pairs)[-10:]
        yield max_10, None


    def steps(self):
        return [
            MRStep(mapper = self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_words)
        ]

if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting Q2.py


In [35]:
import dataset
import pandas as pd

db = dataset.connect('sqlite:///books.db')
desc = pd.DataFrame([book['description'] for book in db['book_info']])
print(desc.head())
desc.to_csv('desc.tsv', sep = '\t')

                                                   0
0  It's hard to imagine a world without A Light i...
1  "Erotic and absorbing...Written with starling ...
2  Dans une France assez proche de la nôtre, un h...
3  WICKED above her hipbone, GIRL across her hear...
4  From a renowned historian comes a groundbreaki...


In [47]:
!python Q2.py desc.tsv

Using configs in /Users/miao/.mrjob.conf
No configs specified for inline runner
Creating temp directory /var/folders/zx/5c8_d0d54710lk5c5dvskks40000gn/T/Q2.miao.20210514.190652.061955
Running step 1 of 2...
Running step 2 of 2...
job output is in /var/folders/zx/5c8_d0d54710lk5c5dvskks40000gn/T/Q2.miao.20210514.190652.061955/output
Streaming final output from /var/folders/zx/5c8_d0d54710lk5c5dvskks40000gn/T/Q2.miao.20210514.190652.061955/output...
[[2002, "with"], [2147, "that"], [2513, "her"], [3136, "is"], [4348, "in"], [6096, "to"], [7088, "a"], [7882, "of"], [8705, "and"], [13156, "the"]]	null
Removing temp directory /var/folders/zx/5c8_d0d54710lk5c5dvskks40000gn/T/Q2.miao.20210514.190652.061955...
